# SETUP

In [3]:
import requests
import datetime
import dash
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [9]:
# You must enter this information to investigate a user
user = 'rnibhriain'
token = input("Input your access token: ")

repos = requests.get(f'https://api.github.com/users/{user}/repos?simple=yes&per_page=100&page=1',headers={"Authorization": f"{token}",'User-Agent': 'request'})
commits = []
numberOfCommits = []
if (repos.status_code == 200):
    for x in repos.json():
        if (x['language'] != None):
            y = x['name']
            current = requests.get(f'https://api.github.com/repos/{user}/{y}/commits?simple=yes&per_page=100&page=1',headers={"Authorization": f"{token}",'User-Agent': 'request'})
            response_dict = current.json()
            
            if (current.status_code == 200):
                commits.append(current)
                numberOfCommits.append(len(current.json()))
else:
    print('Error fetching from API.', repos)

Error fetching from API. 403


In [10]:
def get_productive_days (user, commits) :
    days = []
    times = []
    users = []
    for x in commits:
        for i in x.json(): 
            if (i['commit']['author']['name'] == user):
                date = i['commit']['author']['date'].replace("T", " ")
                date = date.replace("Z", "")
                temp = pd.Timestamp(date)
                dt = temp.day_name()
                
                days.append(dt)
            
                date = date.split(" ")
                temp = pd.Timestamp(date[1])
                
                hour = temp.hour
                times.append(hour)
                
        
                users.append(i['commit']['author']['name'])             
    
    count1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    counts = []
    for i in times:
        count1[i] = count1[i] + 1
    
    for i in times:
        counts.append(count1[i])
    df2 = pd.DataFrame({
        "Days": days,
        "Times": times,
        "User" : users,
        "Count": counts
    })
    return df2

In [11]:
def languages_used(user, repos, numberOfCommits): 
    sizes = []
    languages=[]
    for i in repos.json():
        print(i['language'])
        if(i['language'] != None):
            sizes.append(i['size'])
            print(i['size'])
            languages.append(i['language'])
    lang = []    
    sums = []
    avg = []
    counts = []
    sums1 = []
    avgCount = []
    for i in languages:
        exists = i in lang
        if (exists ==  False and i != None):
            lang.append(i)
            sums.append(0)
            sums1.append(0)
            avg.append(0)
            counts.append(0)
            avgCount.append(0)
            print(i)
    
    for i in range(len(lang)):
        for x in range(len(languages)):
            if (lang[i] == languages[x]):
                sums[i] = sums[i] + sizes[x]
                sums1[i] = sums1[i] + numberOfCommits[x]
                counts[i] = counts[i] + 1
                
    for i in range(len(avg)):
        avg[i] = (sums[i] / counts[i])
        avgCount[i] = (sums1[i] / counts[i])
        print(lang[i])
        print(avg[i])
        print(avgCount[i])
            
    df = pd.DataFrame({
        "Lang": lang,
        "Size": avg,
        "Commits" : avgCount
    })
    return df

# DASH 

In [12]:
df2 = get_productive_days(user, commits)
df1 = languages_used(user, repos, numberOfCommits)
print(df1)
df1 = df1.sort_values("Size")
print(df1)
#set up app

app = dash.Dash()
# create a Dash layout

app = dash.Dash(__name__)

TypeError: string indices must be integers

In [13]:
## Creating the Figures With the Data

In [14]:
fig = px.pie(df1, values='Commits', names='Lang', title=f'Average Commits per Language for {user}',color_discrete_sequence=px.colors.sequential.RdBu)

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig.show()

NameError: name 'df1' is not defined

### Size of Repo per Language

In [ ]:
fig1 = px.pie(df1, values='Size', names='Lang', title=f'Average Size of Repo per Language for {user}', color_discrete_sequence=px.colors.sequential.RdBu)

fig1.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig1.show()

### Size vs Commits Analysis

In [ ]:
figline = px.line(df1, x='Size', y='Commits')
figline.update_traces(line=dict(color = 'white'))

figscatt = px.scatter(df1, x='Size', y='Commits', title='Size vs Commits', color = 'Lang', size = 'Commits' )

figscatt.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig2 = go.Figure(data=figline.data + figscatt.data)

fig2.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text'],
    title = f'Size vs Commits Graph for {user}'
)

fig2.show()

### Productive Hours Graph Setup

In [ ]:
fig3 = px.scatter(df2, x="Days", y="Times", color='Count', size='Count')

fig3.update_xaxes(showgrid=False, categoryorder='array', categoryarray= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

fig3.update_yaxes(showgrid=False, tickvals=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])

fig3.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text'],
    title = f'Commits Per Day for {user}'
)


In [ ]:
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Github Metric Visualisation',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    
    html.H3(
        children='Commits for a Language',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.P(
        children='This piechart shows the average commits to a repository per language',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    dcc.Graph(
        id='example-graph-1',
        figure=fig
    ),
    html.H3(
        children='Size for a Language',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.P(
        children='This piechart shows the average size of a repository per language',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    dcc.Graph(
        id='example-graph-2',
        figure=fig1
    ),
    
    html.H3(
        children='Size vs Commits',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.P(
        children='This graph shows an analysis of the size of repositories vs the number of commits, this should give an indication of what languages the user codes in most often.',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
     html.P(
        children='This would be expected to be linear as the size of the repository would indicate more commits but it is not as can be seen here.',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    dcc.Graph(
        id='example-graph-3',
        figure=fig2
    ),
    html.H3(
        children='Productivity Analysis',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.P(
        children='This graph shows what time of the day the user commits on each day of the week using up to 100 repositories for a user, and up to 100 commits for each repository',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.P(
        children='This will give an indicator of what hours the user is active and most productive.',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    dcc.Graph(
        id='example-graph-4',
        figure=fig3
    )
])

In [ ]:
#run server
app.run_server()